In [2]:
from pathlib import Path
import pandas as pd
import h5py

In [7]:
meta_data = pd.read_csv(Path("data/BraTS2020_training_data/meta_data.csv"))
name_mapping = pd.read_csv(Path("data/BraTS2020_training_data/name_mapping.csv"))
survival_info = pd.read_csv(Path("data/BraTS2020_training_data/survival_info.csv"))

In [8]:
meta_data.head()

,slice_path,target,volume,slice
0,/content/data/volume_41_slice_0.h5,0,41,0
1,/content/data/volume_41_slice_1.h5,0,41,1
2,/content/data/volume_41_slice_2.h5,0,41,2
3,/content/data/volume_41_slice_3.h5,0,41,3
4,/content/data/volume_41_slice_4.h5,0,41,4


In [9]:
name_mapping.head()

,Grade,BraTS_2017_subject_ID,BraTS_2018_subject_ID,TCGA_TCIA_subject_ID,BraTS_2019_subject_ID,BraTS_2020_subject_ID
0,HGG,Brats17_CBICA_AAB_1,Brats18_CBICA_AAB_1,NaN,BraTS19_CBICA_AAB_1,BraTS20_Training_001
1,HGG,Brats17_CBICA_AAG_1,Brats18_CBICA_AAG_1,NaN,BraTS19_CBICA_AAG_1,BraTS20_Training_002
2,HGG,Brats17_CBICA_AAL_1,Brats18_CBICA_AAL_1,NaN,BraTS19_CBICA_AAL_1,BraTS20_Training_003
3,HGG,Brats17_CBICA_AAP_1,Brats18_CBICA_AAP_1,NaN,BraTS19_CBICA_AAP_1,BraTS20_Training_004
4,HGG,Brats17_CBICA_ABB_1,Brats18_CBICA_ABB_1,NaN,BraTS19_CBICA_ABB_1,BraTS20_Training_005


In [10]:
survival_info.head()

,Brats20ID,Age,Survival_days,Extent_of_Resection
0,BraTS20_Training_001,60.463,289,GTR
1,BraTS20_Training_002,52.263,616,GTR
2,BraTS20_Training_003,54.301,464,GTR
3,BraTS20_Training_004,39.068,788,GTR
4,BraTS20_Training_005,68.493,465,GTR


In [11]:
survival_info.hist(column="Survival_days")

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.